In [1]:
from copy import deepcopy

import pandas as pd

from BitBoard import osero
from osero_learn import learn

PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

In [2]:
eva = [[
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
] for i in range(2)]

In [3]:
df = pd.DataFrame({})

check_point = [i for i in range(5, 64, 5)]

for i in range(10):
    print("\r[" + "#" * (i+1) + " " * (10-i+1) + "]", end="")
    for black in PLAY_WAY:
        for white in PLAY_WAY:
            run = learn(\
                black,
                white,
                check_point=check_point,
                seed_num=i,
                eva=eva
            )
            data = run.play()
            df = df.append(data, ignore_index=True)

print("\r[" + "#" * 10 + "]")

[##########]


In [4]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge

x_data = df.drop(["turn", "last_score"], axis=1)
y_data = df[["turn_num", "last_score"]]

turn_vari = df["turn_num"].unique()

In [11]:
def two_plot(x, y, xlabel, ylabel, title, save_dir):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x, y[0], label="train MAE")
    plt.plot(x, y[1], label="test MAE")
    plt.minorticks_on()
    plt.grid(which="major")
    plt.grid(which="minor", linestyle="--")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(save_dir + "/" + title)
    plt.clf()
    plt.close()

In [12]:
alpha_arr = [i * 0.1 for i in range(1, 10)] + [i for i in range(1, 31)]

train_MAE = []
test_MAE = []

for alpha in alpha_arr:
    train_MAE.append([])
    test_MAE.append([])

    for turn_num in turn_vari:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model = Ridge(alpha=alpha, random_state=0)
        model.fit(x_train, y_train)

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

train_MAE = np.array(train_MAE)
test_MAE = np.array(test_MAE)

y_MAE = [[], []]

for i in range(len(train_MAE)):
    y_MAE[0].append(train_MAE[i].mean())
    y_MAE[1].append(test_MAE[i].mean())

two_plot(alpha_arr, y_MAE, "alpha", "MAE", "MAE each alpha", "fig")

In [16]:
max_iter_arr = [i * 1000 for i in range(1, 31)]

train_MAE = []
test_MAE = []

for max_iter in max_iter_arr:
    train_MAE.append([])
    test_MAE.append([])

    for turn_num in turn_vari:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model = Ridge(max_iter=max_iter, random_state=0)
        model.fit(x_train, y_train)

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

train_MAE = np.array(train_MAE)
test_MAE = np.array(test_MAE)

y_MAE = [[], []]

for i in range(len(train_MAE)):
    y_MAE[0].append(train_MAE[i].mean())
    y_MAE[1].append(test_MAE[i].mean())

two_plot(max_iter_arr, y_MAE, "max iter", "MAE", "MAE each max iter", "fig")

In [17]:
tol_arr = [10**-i for i in range(1, 21)]

train_MAE = []
test_MAE = []

for tol in tol_arr:
    train_MAE.append([])
    test_MAE.append([])

    for turn_num in turn_vari:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model = Ridge(tol=tol, random_state=0)
        model.fit(x_train, y_train)

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

train_MAE = np.array(train_MAE)
test_MAE = np.array(test_MAE)

y_MAE = [[], []]

for i in range(len(train_MAE)):
    y_MAE[0].append(train_MAE[i].mean())
    y_MAE[1].append(test_MAE[i].mean())

two_plot(tol_arr, y_MAE, "tol", "MAE", "MAE each tol", "fig")

In [15]:
solver_arr = ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga", "lbfgs"]

train_MAE = []
test_MAE = []

for solver in solver_arr:
    train_MAE.append([])
    test_MAE.append([])

    for turn_num in turn_vari:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        if solver == "lbfgs":
            model = Ridge(solver=solver, positive=True, random_state=0)
        else:
            model = Ridge(solver=solver, random_state=0)
        model.fit(x_train, y_train)

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

train_MAE = np.array(train_MAE)
test_MAE = np.array(test_MAE)

y_MAE = [[], []]

for i in range(len(train_MAE)):
    y_MAE[0].append(train_MAE[i].mean())
    y_MAE[1].append(test_MAE[i].mean())

two_plot(solver_arr, y_MAE, "solver", "MAE", "MAE each solver", "fig")

In [18]:
alpha_arr = [i for i in range(3, 13)]

train_MAE = []
test_MAE = []

for alpha in alpha_arr:
    train_MAE.append([])
    test_MAE.append([])

    for turn_num in turn_vari:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model = Ridge(alpha=alpha, solver="lbfgs", positive=True, random_state=0)
        model.fit(x_train, y_train)

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

train_MAE = np.array(train_MAE)
test_MAE = np.array(test_MAE)

y_MAE = [[], []]

for i in range(len(train_MAE)):
    y_MAE[0].append(train_MAE[i].mean())
    y_MAE[1].append(test_MAE[i].mean())

two_plot(alpha_arr, y_MAE, "alpha", "MAE", "MAE each alpha (solver=lbfgs)", "fig")

In [23]:
train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    x_train, x_test, y_train, y_test = train_test_split(\
        x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        random_state=0
    )

    model = Ridge(alpha=4, solver="lbfgs", positive=True, random_state=0)
    model.fit(x_train, y_train)

    train_predict = model.predict(x_train)
    train_MAE.append(mean_absolute_error(train_predict, y_train))
    test_predict = model.predict(x_test)
    test_MAE.append(mean_absolute_error(test_predict, y_test))

two_plot(
    turn_vari,
    [train_MAE, test_MAE],
    "turn",
    "MAE",
    "MAE each turn (alpha=4, solver=lbfgs)",
    "fig"
)